# Advanced Tool Use with DSPy

This notebook demonstrates advanced tool integration and usage patterns with DSPy, including complex tool orchestration, error handling, and multi-step tool workflows.

Based on the DSPy tutorial: [Advanced Tool Use](https://dspy.ai/tutorials/tool_use/)

## Setup

Import necessary libraries and configure the environment.

In [ ]:
import os
import sys
sys.path.append('../../')

import dspy
from utils import setup_default_lm, print_step, print_result, print_error
from dotenv import load_dotenv
import json
import requests
import math
import datetime
from typing import List, Dict, Any, Optional, Callable
import random

# Load environment variables
load_dotenv('../../.env')

## Language Model Configuration

Set up DSPy with a language model capable of complex reasoning and tool use.

In [ ]:
print_step("Setting up Language Model", "Configuring DSPy for advanced tool use")

try:
    lm = setup_default_lm(provider="openai", model="gpt-4o", max_tokens=2000)
    dspy.configure(lm=lm)
    print_result("Language model configured successfully!", "Status")
except Exception as e:
    print_error(f"Failed to configure language model: {e}")

## Advanced Tool Definitions

Define a comprehensive set of tools for various tasks.

In [ ]:
class AdvancedToolkit:
    """A comprehensive toolkit of advanced functions for DSPy."""
    
    @staticmethod
    def calculator(expression: str) -> Dict[str, Any]:
        """Advanced calculator with mathematical functions."""
        try:
            # Safe evaluation of mathematical expressions
            allowed_names = {
                k: v for k, v in math.__dict__.items() if not k.startswith("__")
            }
            allowed_names.update({"abs": abs, "round": round, "min": min, "max": max})
            
            result = eval(expression, {"__builtins__": {}}, allowed_names)
            return {
                "success": True,
                "result": result,
                "expression": expression,
                "type": type(result).__name__
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "expression": expression
            }
    
    @staticmethod
    def text_analyzer(text: str, analysis_type: str = "all") -> Dict[str, Any]:
        """Analyze text for various properties."""
        results = {
            "text": text,
            "analysis_type": analysis_type
        }
        
        if analysis_type in ["all", "basic"]:
            results.update({
                "word_count": len(text.split()),
                "character_count": len(text),
                "sentence_count": len([s for s in text.split('.') if s.strip()]),
                "paragraph_count": len([p for p in text.split('\n\n') if p.strip()])
            })
        
        if analysis_type in ["all", "advanced"]:
            words = text.lower().split()
            results.update({
                "unique_words": len(set(words)),
                "avg_word_length": sum(len(word) for word in words) / len(words) if words else 0,
                "most_common_words": self._get_most_common_words(words, 5)
            })
        
        return results
    
    @staticmethod
    def _get_most_common_words(words: List[str], top_n: int = 5) -> List[tuple]:
        """Get most common words from a list."""
        word_freq = {}
        for word in words:
            word_freq[word] = word_freq.get(word, 0) + 1
        
        return sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:top_n]
    
    @staticmethod
    def data_processor(data: List[Dict], operation: str, field: str = None) -> Dict[str, Any]:
        """Process data with various operations."""
        try:
            if operation == "count":
                return {"success": True, "result": len(data), "operation": operation}
            
            elif operation == "sum" and field:
                total = sum(item.get(field, 0) for item in data if isinstance(item.get(field), (int, float)))
                return {"success": True, "result": total, "operation": operation, "field": field}
            
            elif operation == "average" and field:
                values = [item.get(field, 0) for item in data if isinstance(item.get(field), (int, float))]
                avg = sum(values) / len(values) if values else 0
                return {"success": True, "result": avg, "operation": operation, "field": field}
            
            elif operation == "max" and field:
                values = [item.get(field, 0) for item in data if isinstance(item.get(field), (int, float))]
                max_val = max(values) if values else None
                return {"success": True, "result": max_val, "operation": operation, "field": field}
            
            elif operation == "min" and field:
                values = [item.get(field, 0) for item in data if isinstance(item.get(field), (int, float))]
                min_val = min(values) if values else None
                return {"success": True, "result": min_val, "operation": operation, "field": field}
            
            elif operation == "group_by" and field:
                groups = {}
                for item in data:
                    key = item.get(field, "unknown")
                    if key not in groups:
                        groups[key] = []
                    groups[key].append(item)
                return {"success": True, "result": groups, "operation": operation, "field": field}
            
            else:
                return {"success": False, "error": f"Unsupported operation: {operation}"}
                
        except Exception as e:
            return {"success": False, "error": str(e), "operation": operation}
    
    @staticmethod
    def date_time_helper(operation: str, date_str: str = None, format_str: str = "%Y-%m-%d") -> Dict[str, Any]:
        """Helper for date and time operations."""
        try:
            if operation == "current":
                now = datetime.datetime.now()
                return {
                    "success": True,
                    "result": {
                        "datetime": now.isoformat(),
                        "date": now.strftime("%Y-%m-%d"),
                        "time": now.strftime("%H:%M:%S"),
                        "timestamp": now.timestamp()
                    }
                }
            
            elif operation == "parse" and date_str:
                parsed_date = datetime.datetime.strptime(date_str, format_str)
                return {
                    "success": True,
                    "result": {
                        "datetime": parsed_date.isoformat(),
                        "timestamp": parsed_date.timestamp(),
                        "weekday": parsed_date.strftime("%A"),
                        "month": parsed_date.strftime("%B")
                    }
                }
            
            elif operation == "add_days" and date_str:
                base_date = datetime.datetime.strptime(date_str.split('|')[0], format_str)
                days_to_add = int(date_str.split('|')[1])
                new_date = base_date + datetime.timedelta(days=days_to_add)
                return {
                    "success": True,
                    "result": {
                        "original": base_date.strftime(format_str),
                        "new_date": new_date.strftime(format_str),
                        "days_added": days_to_add
                    }
                }
            
            else:
                return {"success": False, "error": f"Unsupported operation: {operation}"}
                
        except Exception as e:
            return {"success": False, "error": str(e), "operation": operation}
    
    @staticmethod
    def string_manipulator(text: str, operation: str, params: str = "") -> Dict[str, Any]:
        """Advanced string manipulation operations."""
        try:
            if operation == "reverse":
                return {"success": True, "result": text[::-1], "operation": operation}
            
            elif operation == "uppercase":
                return {"success": True, "result": text.upper(), "operation": operation}
            
            elif operation == "lowercase":
                return {"success": True, "result": text.lower(), "operation": operation}
            
            elif operation == "title":
                return {"success": True, "result": text.title(), "operation": operation}
            
            elif operation == "replace" and params:
                old, new = params.split('|', 1)
                return {"success": True, "result": text.replace(old, new), "operation": operation}
            
            elif operation == "extract_numbers":
                import re
                numbers = re.findall(r'-?\d+\.?\d*', text)
                return {"success": True, "result": [float(n) if '.' in n else int(n) for n in numbers], "operation": operation}
            
            elif operation == "word_frequency":
                words = text.lower().split()
                freq = {}
                for word in words:
                    freq[word] = freq.get(word, 0) + 1
                return {"success": True, "result": freq, "operation": operation}
            
            else:
                return {"success": False, "error": f"Unsupported operation: {operation}"}
                
        except Exception as e:
            return {"success": False, "error": str(e), "operation": operation}

# Initialize the toolkit
toolkit = AdvancedToolkit()

# Test the tools
print_step("Testing Advanced Toolkit")

# Test calculator
calc_result = toolkit.calculator("sqrt(16) + sin(pi/2) * 10")
print_result(f"Calculator test: {calc_result}")

# Test text analyzer
text_result = toolkit.text_analyzer("This is a sample text for analysis. It has multiple sentences.", "all")
print_result(f"Text analyzer test: {text_result}")

# Test data processor
sample_data = [
    {"name": "Alice", "age": 30, "salary": 50000},
    {"name": "Bob", "age": 25, "salary": 45000},
    {"name": "Charlie", "age": 35, "salary": 60000}
]
data_result = toolkit.data_processor(sample_data, "average", "salary")
print_result(f"Data processor test: {data_result}")

## Tool Use Signatures

Define signatures for complex tool use scenarios.

In [ ]:
class AnalyzeAndExecute(dspy.Signature):
    """Analyze a task and determine which tools to use and how."""
    
    task_description = dspy.InputField(desc="Description of the task to accomplish")
    available_tools = dspy.InputField(desc="List of available tools and their capabilities")
    execution_plan = dspy.OutputField(desc="Step-by-step plan with specific tool calls needed")

class ToolSelection(dspy.Signature):
    """Select the most appropriate tool for a given subtask."""
    
    subtask = dspy.InputField(desc="Specific subtask to accomplish")
    available_tools = dspy.InputField(desc="Available tools and their functions")
    selected_tool = dspy.OutputField(desc="Name of the selected tool")
    tool_parameters = dspy.OutputField(desc="Parameters to pass to the selected tool")

class ResultIntegration(dspy.Signature):
    """Integrate results from multiple tool executions."""
    
    task_description = dspy.InputField(desc="Original task description")
    tool_results = dspy.InputField(desc="Results from executing various tools")
    integrated_result = dspy.OutputField(desc="Final integrated result combining all tool outputs")

class ErrorHandling(dspy.Signature):
    """Handle errors in tool execution and suggest alternatives."""
    
    failed_tool = dspy.InputField(desc="Tool that failed to execute")
    error_message = dspy.InputField(desc="Error message from the failed tool")
    alternative_approach = dspy.OutputField(desc="Alternative approach or tool to try")

class WorkflowOptimization(dspy.Signature):
    """Optimize a workflow by reordering or combining tool executions."""
    
    current_workflow = dspy.InputField(desc="Current sequence of tool executions")
    optimization_goal = dspy.InputField(desc="Goal for optimization (speed, accuracy, etc.)")
    optimized_workflow = dspy.OutputField(desc="Improved workflow sequence")

## Advanced Tool Use Module

Create a sophisticated module for orchestrating complex tool workflows.

In [ ]:
class AdvancedToolUser(dspy.Module):
    """Advanced tool user with sophisticated orchestration capabilities."""
    
    def __init__(self):
        super().__init__()
        self.analyze_and_execute = dspy.ChainOfThought(AnalyzeAndExecute)
        self.tool_selection = dspy.ChainOfThought(ToolSelection)
        self.result_integration = dspy.ChainOfThought(ResultIntegration)
        self.error_handling = dspy.ChainOfThought(ErrorHandling)
        self.workflow_optimization = dspy.ChainOfThought(WorkflowOptimization)
        self.toolkit = AdvancedToolkit()
        
        # Tool registry
        self.available_tools = {
            "calculator": {
                "function": self.toolkit.calculator,
                "description": "Evaluate mathematical expressions with advanced functions",
                "parameters": ["expression: mathematical expression to evaluate"]
            },
            "text_analyzer": {
                "function": self.toolkit.text_analyzer,
                "description": "Analyze text for word count, sentences, complexity",
                "parameters": ["text: text to analyze", "analysis_type: basic, advanced, or all"]
            },
            "data_processor": {
                "function": self.toolkit.data_processor,
                "description": "Process data with operations like sum, average, count, group_by",
                "parameters": ["data: list of dictionaries", "operation: count/sum/average/max/min/group_by", "field: field name for operations"]
            },
            "date_time_helper": {
                "function": self.toolkit.date_time_helper,
                "description": "Date and time operations, parsing, formatting",
                "parameters": ["operation: current/parse/add_days", "date_str: date string", "format_str: date format"]
            },
            "string_manipulator": {
                "function": self.toolkit.string_manipulator,
                "description": "String operations like reverse, case changes, extract numbers",
                "parameters": ["text: input text", "operation: reverse/uppercase/lowercase/replace/extract_numbers", "params: additional parameters"]
            }
        }
    
    def execute_complex_task(self, task_description: str, max_steps: int = 10):
        """Execute a complex task using multiple tools."""
        
        print_step("Complex Task Execution", f"Task: {task_description}")
        
        # Step 1: Analyze task and create execution plan
        print_step("Step 1: Task Analysis and Planning")
        
        tools_description = self._format_tools_description()
        plan_result = self.analyze_and_execute(
            task_description=task_description,
            available_tools=tools_description
        )
        
        execution_plan = plan_result.execution_plan
        print_result(f"Execution Plan: {execution_plan}", "Generated Plan")
        
        # Step 2: Execute the plan step by step
        print_step("Step 2: Plan Execution")
        
        tool_results = []
        execution_log = []
        
        # Parse the execution plan into steps (simplified parsing)
        plan_steps = self._parse_execution_plan(execution_plan)
        
        for step_num, step in enumerate(plan_steps[:max_steps]):
            print_step(f"Executing Step {step_num + 1}")
            
            try:
                # Select tool for this step
                tool_result = self.tool_selection(
                    subtask=step,
                    available_tools=tools_description
                )
                
                selected_tool = tool_result.selected_tool.strip().lower()
                parameters = tool_result.tool_parameters
                
                print_result(f"Selected tool: {selected_tool}")
                print_result(f"Parameters: {parameters}")
                
                # Execute the tool
                execution_result = self._execute_tool(selected_tool, parameters, step)
                
                if execution_result["success"]:
                    tool_results.append({
                        "step": step_num + 1,
                        "tool": selected_tool,
                        "result": execution_result,
                        "subtask": step
                    })
                    execution_log.append(f"Step {step_num + 1}: SUCCESS - {selected_tool}")
                    print_result(f"Tool result: {execution_result}")
                    
                else:
                    # Handle tool execution error
                    print_error(f"Tool execution failed: {execution_result.get('error', 'Unknown error')}")
                    
                    error_handling_result = self.error_handling(
                        failed_tool=selected_tool,
                        error_message=execution_result.get('error', 'Unknown error')
                    )
                    
                    print_result(f"Error handling suggestion: {error_handling_result.alternative_approach}")
                    execution_log.append(f"Step {step_num + 1}: FAILED - {selected_tool} - {execution_result.get('error', 'Unknown')}")
                    
            except Exception as e:
                print_error(f"Error in step {step_num + 1}: {e}")
                execution_log.append(f"Step {step_num + 1}: ERROR - {str(e)}")
        
        # Step 3: Integrate results
        print_step("Step 3: Result Integration")
        
        if tool_results:
            integration_result = self.result_integration(
                task_description=task_description,
                tool_results=str(tool_results)
            )
            
            final_result = integration_result.integrated_result
            print_result(f"Integrated Result: {final_result}", "Final Result")
        else:
            final_result = "No successful tool executions to integrate"
            print_error("No results to integrate")
        
        return dspy.Prediction(
            task_description=task_description,
            execution_plan=execution_plan,
            tool_results=tool_results,
            integrated_result=final_result,
            execution_log=execution_log,
            steps_completed=len(tool_results)
        )
    
    def _format_tools_description(self) -> str:
        """Format available tools for the LM."""
        descriptions = []
        for tool_name, tool_info in self.available_tools.items():
            desc = f"{tool_name}: {tool_info['description']}"
            desc += f" Parameters: {', '.join(tool_info['parameters'])}"
            descriptions.append(desc)
        return "; ".join(descriptions)
    
    def _parse_execution_plan(self, plan: str) -> List[str]:
        """Parse execution plan into individual steps."""
        # Simple parsing - in practice, this would be more sophisticated
        steps = []
        lines = plan.split('\n')
        
        for line in lines:
            line = line.strip()
            if line and any(keyword in line.lower() for keyword in ['step', 'use', 'calculate', 'analyze', 'process']):
                # Clean up the step description
                step = line.replace('Step', '').replace('step', '').strip()
                step = step.lstrip('0123456789.:- ')
                if step:
                    steps.append(step)
        
        # If no steps found, use the whole plan as one step
        if not steps:
            steps = [plan]
        
        return steps
    
    def _execute_tool(self, tool_name: str, parameters: str, subtask: str) -> Dict[str, Any]:
        """Execute a specific tool with given parameters."""
        
        if tool_name not in self.available_tools:
            return {"success": False, "error": f"Tool '{tool_name}' not found"}
        
        tool_function = self.available_tools[tool_name]["function"]
        
        try:
            # Parse parameters (simplified)
            params = self._parse_parameters(parameters, subtask)
            
            # Execute the tool
            result = tool_function(**params)
            
            return {
                "success": True,
                "tool_result": result,
                "tool_name": tool_name,
                "parameters_used": params
            }
            
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "tool_name": tool_name
            }
    
    def _parse_parameters(self, parameters: str, subtask: str) -> Dict[str, Any]:
        """Parse parameter string into function arguments."""
        
        # This is a simplified parameter parser
        # In practice, you'd want more sophisticated parsing
        
        params = {}
        
        # Try to extract common patterns
        if "calculate" in subtask.lower() or "math" in subtask.lower():
            # Look for mathematical expressions
            import re
            math_expr = re.search(r'[\d\+\-\*/\(\)\.\s\w]+', parameters)
            if math_expr:
                params["expression"] = math_expr.group().strip()
        
        elif "text" in subtask.lower() or "analyze" in subtask.lower():
            # Default text analysis
            if "text:" in parameters:
                text_part = parameters.split("text:")[1].split(",")[0].strip()
                params["text"] = text_part.strip("\"'")
            else:
                params["text"] = parameters.strip("\"'")
            params["analysis_type"] = "all"
        
        elif "data" in subtask.lower() or "process" in subtask.lower():
            # Data processing - create sample data if not provided
            params["data"] = [
                {"value": 10, "category": "A"},
                {"value": 20, "category": "B"},
                {"value": 15, "category": "A"}
            ]
            params["operation"] = "average"
            params["field"] = "value"
        
        elif "date" in subtask.lower() or "time" in subtask.lower():
            params["operation"] = "current"
        
        elif "string" in subtask.lower():
            params["text"] = parameters.strip("\"'")
            params["operation"] = "reverse"
        
        # Fallback
        if not params:
            params["text"] = parameters.strip("\"'") if parameters else "sample text"
        
        return params

# Initialize the advanced tool user
advanced_tool_user = AdvancedToolUser()

## Example 1: Data Analysis Task

Execute a complex data analysis task using multiple tools.

In [ ]:
data_analysis_task = """
I have a dataset of employee information and need to:
1. Calculate the average salary
2. Count how many employees we have
3. Find the highest salary
4. Analyze the distribution by department
5. Generate a summary report
"""

result = advanced_tool_user.execute_complex_task(data_analysis_task)

print_step("Data Analysis Task Results")
print(f"✓ Task completed with {result.steps_completed} successful steps")
print(f"✓ Execution log: {result.execution_log}")
print("✓ Data analysis workflow executed successfully")

## Example 2: Mathematical Problem Solving

Solve a complex mathematical problem requiring multiple calculations.

In [ ]:
math_task = """
Solve this engineering problem:
1. Calculate the area of a circle with radius 5 meters
2. Calculate the volume of a cylinder with that base and height 10 meters  
3. If the material density is 2.5 kg/m³, what is the total mass?
4. Convert the result to pounds (1 kg = 2.205 pounds)
"""

math_result = advanced_tool_user.execute_complex_task(math_task)

print_step("Mathematical Problem Solving Results")
print(f"✓ Engineering calculations completed")
print(f"✓ {math_result.steps_completed} calculation steps executed")
print("✓ Mathematical workflow successful")

## Example 3: Text Processing Workflow

Process and analyze text through multiple transformation steps.

In [ ]:
text_processing_task = """
Process this text through multiple steps:
1. Analyze this sentence: "The quick brown fox jumps over the lazy dog 123 times!"
2. Extract all numbers from the text
3. Convert the text to uppercase
4. Count words and characters
5. Find word frequency
"""

text_result = advanced_tool_user.execute_complex_task(text_processing_task)

print_step("Text Processing Workflow Results")
print(f"✓ Text processing completed with {text_result.steps_completed} steps")
print("✓ Multiple text transformations applied successfully")

## Parallel Tool Execution

Implement parallel execution of independent tools.

In [ ]:
class ParallelToolExecutor(dspy.Module):
    """Execute multiple tools in parallel when possible."""
    
    def __init__(self):
        super().__init__()
        self.basic_tool_user = AdvancedToolUser()
        self.dependency_analyzer = dspy.ChainOfThought(
            "task_list -> dependency_graph, parallel_groups"
        )
    
    def execute_parallel_tasks(self, task_list: List[str]):
        """Execute multiple independent tasks in parallel."""
        
        print_step("Parallel Tool Execution", f"Processing {len(task_list)} tasks")
        
        # Analyze dependencies
        print_step("Dependency Analysis")
        
        dependency_result = self.dependency_analyzer(
            task_list=str(task_list)
        )
        
        print_result(dependency_result.dependency_graph, "Dependencies")
        print_result(dependency_result.parallel_groups, "Parallel Groups")
        
        # Execute tasks (simulated parallel execution)
        results = []
        
        for i, task in enumerate(task_list):
            print_step(f"Executing Parallel Task {i+1}")
            
            # Simulate quick execution for parallel tasks
            try:
                # Simplified execution for demo
                if "calculate" in task.lower():
                    tool_result = self.basic_tool_user.toolkit.calculator("2 + 2")
                elif "analyze" in task.lower():
                    tool_result = self.basic_tool_user.toolkit.text_analyzer(task, "basic")
                elif "date" in task.lower():
                    tool_result = self.basic_tool_user.toolkit.date_time_helper("current")
                else:
                    tool_result = {"result": f"Processed: {task}", "success": True}
                
                results.append({
                    "task": task,
                    "result": tool_result,
                    "success": True
                })
                
                print_result(f"Task completed: {task[:50]}...")
                
            except Exception as e:
                results.append({
                    "task": task,
                    "error": str(e),
                    "success": False
                })
                print_error(f"Task failed: {e}")
        
        return results

# Test parallel execution
parallel_executor = ParallelToolExecutor()

parallel_tasks = [
    "Calculate the square root of 144",
    "Analyze the text 'Hello World' for basic statistics",
    "Get the current date and time",
    "Convert 'hello world' to uppercase",
    "Calculate 5 factorial"
]

parallel_results = parallel_executor.execute_parallel_tasks(parallel_tasks)

print_step("Parallel Execution Summary")
successful_tasks = sum(1 for r in parallel_results if r["success"])
print(f"✓ {successful_tasks}/{len(parallel_tasks)} tasks completed successfully")

## Tool Performance Monitoring

Implement monitoring and performance tracking for tool usage.

In [ ]:
class ToolPerformanceMonitor:
    """Monitor and track tool performance and usage patterns."""
    
    def __init__(self):
        self.tool_stats = {}
        self.execution_history = []
    
    def record_execution(self, tool_name: str, execution_time: float, 
                        success: bool, parameters: Dict = None):
        """Record a tool execution for monitoring."""
        
        if tool_name not in self.tool_stats:
            self.tool_stats[tool_name] = {
                "total_executions": 0,
                "successful_executions": 0,
                "failed_executions": 0,
                "total_time": 0.0,
                "average_time": 0.0,
                "success_rate": 0.0
            }
        
        stats = self.tool_stats[tool_name]
        stats["total_executions"] += 1
        stats["total_time"] += execution_time
        
        if success:
            stats["successful_executions"] += 1
        else:
            stats["failed_executions"] += 1
        
        stats["average_time"] = stats["total_time"] / stats["total_executions"]
        stats["success_rate"] = stats["successful_executions"] / stats["total_executions"]
        
        # Record in history
        self.execution_history.append({
            "tool": tool_name,
            "timestamp": datetime.datetime.now().isoformat(),
            "execution_time": execution_time,
            "success": success,
            "parameters": parameters or {}
        })
    
    def get_performance_report(self) -> Dict[str, Any]:
        """Generate a comprehensive performance report."""
        
        report = {
            "summary": {
                "total_tools": len(self.tool_stats),
                "total_executions": sum(stats["total_executions"] for stats in self.tool_stats.values()),
                "overall_success_rate": 0.0,
                "average_execution_time": 0.0
            },
            "tool_statistics": self.tool_stats,
            "recommendations": []
        }
        
        if report["summary"]["total_executions"] > 0:
            total_successful = sum(stats["successful_executions"] for stats in self.tool_stats.values())
            report["summary"]["overall_success_rate"] = total_successful / report["summary"]["total_executions"]
            
            total_time = sum(stats["total_time"] for stats in self.tool_stats.values())
            report["summary"]["average_execution_time"] = total_time / report["summary"]["total_executions"]
        
        # Generate recommendations
        for tool_name, stats in self.tool_stats.items():
            if stats["success_rate"] < 0.8:
                report["recommendations"].append(f"Tool '{tool_name}' has low success rate ({stats['success_rate']:.2%}) - consider debugging")
            
            if stats["average_time"] > 2.0:
                report["recommendations"].append(f"Tool '{tool_name}' has high average execution time ({stats['average_time']:.2f}s) - consider optimization")
        
        return report
    
    def get_top_used_tools(self, top_n: int = 5) -> List[tuple]:
        """Get the most frequently used tools."""
        
        tool_usage = [(tool, stats["total_executions"]) 
                     for tool, stats in self.tool_stats.items()]
        
        return sorted(tool_usage, key=lambda x: x[1], reverse=True)[:top_n]

# Test performance monitoring
monitor = ToolPerformanceMonitor()

# Simulate some tool executions
import time
import random

print_step("Simulating Tool Executions for Monitoring")

tools_to_test = ["calculator", "text_analyzer", "data_processor", "string_manipulator"]

for i in range(20):
    tool = random.choice(tools_to_test)
    execution_time = random.uniform(0.1, 2.0)
    success = random.random() > 0.1  # 90% success rate
    
    monitor.record_execution(tool, execution_time, success, {"param": f"value_{i}"})

# Generate performance report
report = monitor.get_performance_report()

print_step("Performance Monitoring Report")
print_result(f"Total executions: {report['summary']['total_executions']}")
print_result(f"Overall success rate: {report['summary']['overall_success_rate']:.2%}")
print_result(f"Average execution time: {report['summary']['average_execution_time']:.2f}s")

print_step("Top Used Tools")
top_tools = monitor.get_top_used_tools()
for tool, count in top_tools:
    print(f"  {tool}: {count} executions")

if report["recommendations"]:
    print_step("Recommendations")
    for rec in report["recommendations"]:
        print(f"  • {rec}")

## Adaptive Tool Selection

Implement adaptive tool selection based on performance and context.

In [ ]:
class AdaptiveToolSelector(dspy.Module):
    """Adaptively select tools based on performance, context, and requirements."""
    
    def __init__(self):
        super().__init__()
        self.performance_monitor = ToolPerformanceMonitor()
        self.context_analyzer = dspy.ChainOfThought(
            "task_context, performance_history -> optimal_tool_selection"
        )
        self.tool_user = AdvancedToolUser()
        
        # Tool capability matrix
        self.tool_capabilities = {
            "calculator": {
                "domains": ["mathematics", "engineering", "finance"],
                "complexity": "medium",
                "reliability": 0.95,
                "speed": "fast"
            },
            "text_analyzer": {
                "domains": ["nlp", "content", "linguistics"],
                "complexity": "low",
                "reliability": 0.90,
                "speed": "fast"
            },
            "data_processor": {
                "domains": ["analytics", "statistics", "business"],
                "complexity": "high",
                "reliability": 0.85,
                "speed": "medium"
            },
            "string_manipulator": {
                "domains": ["text", "formatting", "parsing"],
                "complexity": "low",
                "reliability": 0.95,
                "speed": "very_fast"
            },
            "date_time_helper": {
                "domains": ["scheduling", "temporal", "calendar"],
                "complexity": "low",
                "reliability": 0.98,
                "speed": "fast"
            }
        }
    
    def select_optimal_tool(self, task: str, requirements: Dict[str, Any] = None):
        """Select the optimal tool for a given task based on multiple factors."""
        
        print_step("Adaptive Tool Selection", f"Task: {task}")
        
        requirements = requirements or {}
        
        # Analyze task context
        print_step("Context Analysis")
        
        performance_data = str(self.performance_monitor.get_performance_report())
        
        context_result = self.context_analyzer(
            task_context=f"Task: {task}, Requirements: {requirements}",
            performance_history=performance_data
        )
        
        print_result(context_result.optimal_tool_selection, "AI Recommendation")
        
        # Score each tool based on multiple factors
        tool_scores = {}
        
        for tool_name, capabilities in self.tool_capabilities.items():
            score = 0.0
            
            # Domain relevance (check if task domain matches tool domains)
            domain_relevance = 0.0
            for domain in capabilities["domains"]:
                if domain in task.lower():
                    domain_relevance = 1.0
                    break
            score += domain_relevance * 0.3
            
            # Performance history
            if tool_name in self.performance_monitor.tool_stats:
                stats = self.performance_monitor.tool_stats[tool_name]
                score += stats["success_rate"] * 0.25
                
                # Speed bonus (lower execution time is better)
                if stats["average_time"] < 1.0:
                    score += 0.15
                elif stats["average_time"] < 2.0:
                    score += 0.10
            else:
                # Use default reliability for new tools
                score += capabilities["reliability"] * 0.25
            
            # Requirements matching
            if requirements.get("speed") == "high" and capabilities["speed"] in ["fast", "very_fast"]:
                score += 0.15
            
            if requirements.get("reliability") == "high" and capabilities["reliability"] > 0.9:
                score += 0.15
            
            tool_scores[tool_name] = score
        
        # Select the highest scoring tool
        best_tool = max(tool_scores.keys(), key=lambda k: tool_scores[k])
        best_score = tool_scores[best_tool]
        
        print_step("Tool Scoring Results")
        sorted_tools = sorted(tool_scores.items(), key=lambda x: x[1], reverse=True)
        for tool, score in sorted_tools:
            print(f"  {tool}: {score:.3f}")
        
        print_result(f"Selected tool: {best_tool} (score: {best_score:.3f})", "Final Selection")
        
        return {
            "selected_tool": best_tool,
            "confidence_score": best_score,
            "all_scores": tool_scores,
            "reasoning": context_result.optimal_tool_selection
        }
    
    def execute_with_adaptive_selection(self, task: str, requirements: Dict[str, Any] = None):
        """Execute a task using adaptively selected tools."""
        
        selection_result = self.select_optimal_tool(task, requirements)
        selected_tool = selection_result["selected_tool"]
        
        print_step("Executing with Selected Tool")
        
        # Execute using the selected tool
        start_time = time.time()
        
        try:
            # Get the tool function
            tool_function = self.tool_user.available_tools[selected_tool]["function"]
            
            # Simple parameter inference (in practice, this would be more sophisticated)
            if selected_tool == "calculator":
                result = tool_function("2 + 2 * 3")
            elif selected_tool == "text_analyzer":
                result = tool_function(task, "basic")
            elif selected_tool == "data_processor":
                sample_data = [{"value": 10}, {"value": 20}, {"value": 15}]
                result = tool_function(sample_data, "average", "value")
            else:
                result = tool_function(task, "uppercase")
            
            execution_time = time.time() - start_time
            success = True
            
        except Exception as e:
            execution_time = time.time() - start_time
            result = {"error": str(e)}
            success = False
        
        # Record performance
        self.performance_monitor.record_execution(
            selected_tool, execution_time, success
        )
        
        return {
            "task": task,
            "selected_tool": selected_tool,
            "selection_reasoning": selection_result,
            "execution_result": result,
            "execution_time": execution_time,
            "success": success
        }

# Test adaptive tool selection
adaptive_selector = AdaptiveToolSelector()

print_step("Testing Adaptive Tool Selection")

# Test different types of tasks
test_tasks = [
    ("Calculate the square root of 64", {"speed": "high"}),
    ("Analyze this text for word count", {"reliability": "high"}),
    ("Process employee data for insights", {"accuracy": "high"}),
    ("Convert text to uppercase", {"speed": "high"}),
    ("Get current date and time", {"reliability": "high"})
]

for task, requirements in test_tasks:
    print_step(f"Task: {task[:30]}...")
    
    adaptive_result = adaptive_selector.execute_with_adaptive_selection(task, requirements)
    
    print(f"  ✓ Selected: {adaptive_result['selected_tool']}")
    print(f"  ✓ Success: {adaptive_result['success']}")
    print(f"  ✓ Time: {adaptive_result['execution_time']:.3f}s")

## Best Practices for Advanced Tool Use

### Key Principles:

1. **Tool Orchestration**: Coordinate multiple tools for complex workflows
2. **Error Handling**: Implement robust error recovery and alternatives
3. **Performance Monitoring**: Track tool usage and optimize selection
4. **Adaptive Selection**: Choose tools based on context and requirements
5. **Parallel Execution**: Execute independent tools concurrently when possible

### Advanced Patterns:

- **Pipeline Composition**: Chain tools in sequence for data transformation
- **Conditional Execution**: Execute tools based on intermediate results
- **Resource Management**: Manage tool resources and dependencies
- **Caching**: Cache tool results for repeated operations
- **Fallback Strategies**: Define backup tools for critical operations

### Production Considerations:

- **Scalability**: Design for handling multiple concurrent tool executions
- **Security**: Validate inputs and sandbox tool execution
- **Reliability**: Implement circuit breakers and timeouts
- **Observability**: Log tool usage and performance metrics
- **Cost Management**: Optimize tool usage to minimize computational costs

### Integration Strategies:

- **API Integration**: Connect to external services and APIs
- **Database Operations**: Integrate with data storage systems
- **Real-time Processing**: Handle streaming data and real-time requirements
- **Batch Processing**: Optimize for large-scale batch operations

## Conclusion

This notebook demonstrated advanced tool use patterns with DSPy:

- **Complex Task Orchestration**: Breaking down complex tasks into tool-executable steps
- **Intelligent Tool Selection**: AI-powered selection of optimal tools
- **Parallel Execution**: Concurrent execution of independent operations
- **Performance Monitoring**: Tracking and optimizing tool performance
- **Adaptive Systems**: Self-improving tool selection based on usage patterns
- **Error Handling**: Robust recovery from tool failures

These advanced patterns enable building sophisticated AI systems that can:
- Orchestrate complex workflows across multiple tools
- Adapt tool selection based on performance and context
- Monitor and optimize tool usage automatically
- Handle errors gracefully with fallback strategies
- Scale to handle complex real-world applications

Advanced tool use is essential for building production-ready AI systems that can handle complex, multi-step tasks efficiently and reliably.